In [7]:
df_valid_X.head(2)

NameError: name 'df_valid_X' is not defined

In [3]:
import pandas as pd
import numpy as np
from glob2 import glob
import json

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords as stp
from nltk.stem.porter import PorterStemmer

import gensim
import gensim.models.word2vec as w2v

from datetime import datetime
from datetime import date
from datetime import timedelta

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

[nltk_data] Downloading package punkt to /Users/june/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/june/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_colwidth', 60)

# TOC
1. Data Import
    1. Feature Extraction
2. Text Vectorization
    2. Word2Vec
    3. Doc2Vec
4. Data Export as CSV files

# Data Import

## Functions

In [5]:
import random

def cross_val_jsons(jsonFiles, isTest):
    # output = [json for json in jsonFiles]
    # print(random.choice(jsonFiles))
    if (isTest == True):
        test = random.choice(jsonFiles)
        jsonFiles.remove(test)
        train = [json for event in jsonFiles for json in event]
        return train, test
    else:
        data = [json for event in jsonFiles for json in event]
        return data

def extract_data(datas):
    tweet_data_frame = []
    data_lists = []
    isRumorLists = []
    for index, dataset in enumerate(datas):
        data_list = []
        isRumorList = []
        count = 0 # help var

        for jsonFile in dataset:
            count+=1
            if jsonFile.find("non-rumours") == -1:
                isRumorList.append(1)
            else:
                isRumorList.append(0)

            with open (jsonFile, 'r') as f:
                for l in f.readlines():
                    if not l.strip (): # skip empty lines
                        continue
                    json_data = json.loads(l)
                    # print (json_data,"\n\n")
                    data_list.append(json_data)

        isRumorLists.append(pd.DataFrame(isRumorList, columns=['isRumor']))
        data_lists.append(data_list)
        tweet_data_frame.append(pd.DataFrame.from_dict(data_list))
    return data_lists, isRumorLists, tweet_data_frame

def printRumor(route):
    if route.find("rumours") == -1:
        print('non-rumors')
    else:
        print('rumor')

In [6]:
def capitalratio(tweet_text):
    uppers = [l for l in tweet_text if l.isupper()]
    capitalratio = len(uppers) / len(tweet_text)
    return capitalratio 

def tweets2tokens(tweet_text):
    # Tokenizing
    urls = []
    tokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+','', tweet_text.lower()))

    # Setting url value (whether the tweet contains http link) and filter http links
    url=0
    for token in tokens:
        if token.startswith('http'):
            url=1
    tokens = [token for token in tokens if not token.startswith('http')]

    ## Stemming
    # porter = PorterStemmer()
    # tokens = [porter.stem(token) for token in tokens]

    # Filtering Stop words
    # from nltk.corpus import stopwords
    # stop_words = set(stopwords.words('english'))
    # tokens = [token for token in tokens if not token in stop_words]

    return tokens, url

def getposcount(tokens):
    postag = []
    poscount = {}
    poscount['Noun']=0
    poscount['Verb']=0
    poscount['Adjective'] = 0
    poscount['Pronoun']=0
    poscount['FirstPersonPronoun']=0
    poscount['SecondPersonPronoun']=0
    poscount['ThirdPersonPronoun']=0
    poscount['Adverb']=0
    poscount['Numeral']=0
    poscount['Conjunction_inj']=0
    poscount['Particle']=0
    poscount['Determiner']=0
    poscount['Modal']=0
    poscount['Whs']=0
    Nouns = {'NN','NNS','NNP','NNPS'}
    Adverbs = {'RB','RBR','RBS'}
    Whs = {'WDT','WP','WRB'} # Composition of wh-determiner(that,what), wh-pronoun(who), wh-adverb(how)
    Verbs={'VB','VBP','VBZ','VBN','VBG','VBD','To'}
    first_person_pronouns=['i','I','me','my','mine','we','us','our','ours'] #'i',
    second_person_pronouns=['you','your','yours']
    third_person_pronouns=['he','she','it','him','her','it','his','hers','its','they','them','their','theirs']

    for word in tokens:
        w_lower=word.lower()
        if w_lower in first_person_pronouns:
            poscount['FirstPersonPronoun']+=1
        elif w_lower in second_person_pronouns:
            poscount['SecondPersonPronoun']+=1
        elif w_lower in third_person_pronouns:
            poscount['ThirdPersonPronoun']+=1
    
    postag = nltk.pos_tag(tokens)
    for g1 in postag:
        if g1[1] in Nouns:
            poscount['Noun'] += 1
        elif g1[1] in Verbs:
            poscount['Verb']+= 1
        elif g1[1]=='ADJ'or g1[1]=='JJ':
            poscount['Adjective']+=1
        elif g1[1]=='PRP' or g1[1]=='PRON' or g1[1]=='PRP$':
            poscount['Pronoun']+=1
        elif g1[1] in Adverbs or g1[1]=='ADV':
            poscount['Adverb']+=1
        elif g1[1]=='CD':
            poscount['Numeral']+=1
        elif g1[1]=='CC' or g1[1]=='IN':
            poscount['Conjunction_inj']+=1
        elif g1[1]=='RP':
            poscount['Particle']+=1
        elif g1[1]=='MD':
            poscount['Modal']+=1
        elif g1[1]=='DT':
            poscount['Determiner']+=1
        elif g1[1] in Whs:
            poscount['Whs']+=1
    return poscount

def contentlength(words):
    wordcount = len(words)
    return wordcount

def extract_urls(entities_dicts):
    if len(entities_dicts) < 1:
        return 0,0,0

    urls = []
    urls_expanded = []

    key = 'url'
    key2 = 'expanded_url'
    # print(len(entities_dict))
    for i in entities_dicts:
        urls.append(i[key])
        urls_expanded.append(i[key2])
    return 1, urls, urls_expanded

def flatten_tweets(tweets):
    """ Flattens out tweet dictionaries so relevant JSON is in a top-level dictionary. """
    tweets_list = []
    total_tokens_l = []

    # Iterate through each tweet
    for tweet_obj in tweets:
        output_f = dict()

        output_f['text']= tweet_obj['text']
        
        urls_dicts = tweet_obj['entities']['urls']
        # print(urls_dicts)

        output_f['hasURL'], output_f['urls'], output_f['urls_expanded'] = extract_urls(urls_dicts)
        
        # print(type(tweet_obj['user']))
        # print(tweet_obj['user'].contains_key('entities'))
        if ('url' in tweet_obj['user']):
            output_f['hasUserURL'] = 1
            output_f['user_url'] = tweet_obj['user']['url']
        elif ('entities' in tweet_obj['user']):
            # output_f['user_entity'] = tweet_obj['user']['entities']['url']['urls']
            # print(tweet_obj['user']['entities']['url']['urls'])
            # output_f['user_url'] = tweet_obj['user']['entities']['expanded_url']
            output_f['hasUserURL'] , _ , output_f['user_url'] = extract_urls(tweet_obj['user']['entities']['url']['urls'])
        else:
            # output_f['user_entity'] = None
            output_f['user_url'] = 0
            output_f['hasUserURL'] = 0

        output_f['text_token'], output_f['isNotOnlyText'] = tweets2tokens(tweet_obj['text'])
        total_tokens_l.extend(output_f['text_token']) # append the tokens to list of total tokens

        '''POS Tagging'''
        pos_dict=getposcount(output_f['text_token'])
        output_f.update(pos_dict)

        output_f['char_count'] = len(output_f['text'])
        output_f['word_count'] = len(output_f['text_token'])

        output_f['has_question'] = "?" in output_f["text"]
        output_f['has_exclaim'] = "!" in output_f["text"]
        output_f['has_period'] = "." in output_f["text"]
    
        ''' User info'''
        # Store the user screen name in 'user-screen_name'
        # output_f['user-screen_name'] = tweet_obj['user']['screen_name']
        
        # Store the user location
        # output_f['user-location'] = tweet_obj['user']['location']

        acc_created = datetime.strptime(tweet_obj['user']['created_at'], '%a %b %d %H:%M:%S %z %Y')
        tweet_created = datetime.strptime(tweet_obj['created_at'], '%a %b %d %H:%M:%S %z %Y')
        age = (tweet_created - acc_created)
        # print(type(timedelta.total_seconds(age)))

        output_f['capital_ratio']=(capitalratio(tweet_obj['text']))

        # features=(capitalratio(data_list[0]['user']))
        output_f['tweet_count'] = np.log10(tweet_obj['user']['statuses_count'])
        output_f['listed_count'] = np.log10(tweet_obj['user']['listed_count'])
        output_f['follow_ratio'] = np.log10(tweet_obj['user']['followers_count'])
        output_f['age'] = int(timedelta.total_seconds(age)/86400)
        output_f['verified'] = tweet_obj['user']['verified']

        tweets_list.append(output_f)

    unk_tokens_l = list(set(total_tokens_l))
    print("Number of total tokens appeared: {}\nNumber of unique tokens appeared: {}\n".format(len(total_tokens_l), len(unk_tokens_l))) # number of tokens and unique tokens

    return tweets_list

## Importing JSON Files and grouping

### PHEME dataset of social media rumours

Used in:
- Arkaitz Zubiaga, Maria Liakata, Rob Procter, Kalina Bontcheva, Peter Tolmie. Crowdsourcing the Annotation of Rumourous Conversations in Social Media. WWW Companion. 2015.
   http://www.zubiaga.org/publications/files/www2015-crowdsourcing.pdf

- Arkaitz Zubiaga, Maria Liakata, Rob Procter, Geraldine Wong Sak Hoi, Peter Tolmie. Analysing How People Orient to and Spread Rumours in Social Media by Looking at Conversational Threads. PLoS ONE 11(3): e0150989. 2016. http://journals.plos.org/plosone/article?id=10.1371%2Fjournal.pone.0150989


In [48]:
charliehebdo_jsons = glob('../pheme-rumour-scheme-dataset/threads/en/charliehebdo/**/source-tweets/*.json') 
ferguson_jsons = glob('../pheme-rumour-scheme-dataset/threads/en/ferguson/**/source-tweets/*.json')
germanwing_scrash_jsons = glob('../pheme-rumour-scheme-dataset/threads/en/germanwings-crash/**/source-tweets/*.json')
ottawashooting_jsons = glob('../pheme-rumour-scheme-dataset/threads/en/ottawashooting/**/source-tweets/*.json')
sydneysiege_jsons = glob('../pheme-rumour-scheme-dataset/threads/en/sydneysiege/**/source-tweets/*.json')
putinmissing_jsons = glob('../pheme-rumour-scheme-dataset/threads/en/putinmissing/**/source-tweets/*.json')
prince_toronto_jsons = glob('../pheme-rumour-scheme-dataset/threads/en/prince-toronto/**/source-tweets/*.json')
ebolaessien_jsons = glob('../pheme-rumour-scheme-dataset/threads/en/ebola-essien/**/source-tweets/*.json')

annotated_files = [charliehebdo_jsons, ferguson_jsons, germanwing_scrash_jsons, ottawashooting_jsons, sydneysiege_jsons, gurlitt_jsons, ebolaessien_jsons, putinmissing_jsons]

# tweet_data_frame = [] # data_lists = [] # isRumorLists = []
train, test = cross_val_jsons(annotated_files, True)
data_lists, isRumorLists, tweet_data_frame = extract_data([train, test[0:100]])
train, test = data_lists[0], data_lists[1]
df_train_y, df_test_y = isRumorLists[0],isRumorLists[1]
print("(Data of Root tweets) Train: {} Test: {}".format(len(train),len(test)))
print("(Data of Root tweets) Train_y: {} Test_y: {}".format(len(isRumorLists[0]),len(isRumorLists[1])))

(Data of Root tweets) Train: 352 Test: 71
(Data of Root tweets) Train_y: 352 Test_y: 71


### PHEME dataset for Rumour Detection and Veracity Classification - extension

Used in:
- 'All-in-one: Multi-task Learning for Rumour Verification'. (https://github.com/kochkinaelena/Multitask4Veracity).

In [42]:
all_jsons = glob('../PHEME/all-rnr-annotated-threads/**/**/source-tweets/*.json') 

# gurlitt_jsons = glob('../PHEME/all-rnr-annotated-threads/gurlitt-all-rnr-threads/**/source-tweets/*.json')
# ebolaessien_jsons = glob('../PHEME/all-rnr-annotated-threads/ebola-essien-all-rnr-threads/**/source-tweets/*.json')
# putinmissing_jsons = glob('../PHEME/all-rnr-annotated-threads/putinmissing-all-rnr-threads/**/source-tweets/*.json')
# added_files = [gurlitt_jsons, ebolaessien_jsons, putinmissing_jsons]

print(len(all_jsons))

19275


In [80]:
charliehebdo_jsons = glob('../PHEME/all-rnr-annotated-threads/charliehebdo-all-rnr-threads/**/source-tweets/*.json') 
ferguson_jsons = glob('../PHEME/all-rnr-annotated-threads/ferguson-all-rnr-threads/**/source-tweets/*.json')
germanwing_scrash_jsons = glob('../PHEME/all-rnr-annotated-threads/germanwings-crash-all-rnr-threads/**/source-tweets/*.json')
ottawashooting_jsons = glob('../PHEME/all-rnr-annotated-threads/ottawashooting-all-rnr-threads/**/source-tweets/*.json')
sydneysiege_jsons = glob('../PHEME/all-rnr-annotated-threads/sydneysiege-all-rnr-threads/**/source-tweets/*.json')
gurlitt_jsons = glob('../PHEME/all-rnr-annotated-threads/gurlitt-all-rnr-threads/**/source-tweets/*.json')
ebolaessien_jsons = glob('../PHEME/all-rnr-annotated-threads/ebola-essien-all-rnr-threads/**/source-tweets/*.json')
putinmissing_jsons = glob('../PHEME/all-rnr-annotated-threads/putinmissing-all-rnr-threads/**/source-tweets/*.json')

# annotated_files = [charliehebdo_jsons, ferguson_jsons, germanwing_scrash_jsons, ottawashooting_jsons, sydneysiege_jsons, gurlitt_jsons, ebolaessien_jsons, putinmissing_jsons]
added_files = [gurlitt_jsons, ebolaessien_jsons, putinmissing_jsons]

In [213]:
# tweet_data_frame = [] # data_lists = [] # isRumorLists = []
valid_data = cross_val_jsons(added_files, False)
data_lists, isRumorLists, tweet_data_frame = extract_data([valid_data])
X_valid = data_lists[0]
y_valid = isRumorLists[0]
print("(Data of Root tweets) Train: {} Test: {}".format(len(X_valid),len(y_valid)))

(Data of Root tweets) Train: 390 Test: 390


## Flatten and extract basic features

In [393]:
df_valid_X = pd.DataFrame(flatten_tweets(X_valid))

Number of total tokens appeared: 5637
Number of unique tokens appeared: 1516



In [394]:
df_valid = pd.concat([df_valid_X,y_valid],axis=1)

In [395]:
df_valid[['has_question', 'has_exclaim', 'has_period','verified']] = df_valid[['has_question', 'has_exclaim', 'has_period','verified']].astype(int)

## Handling inf and NaN value

In [396]:
for dataset in [df_valid]:
    dataset['listed_count'].replace([np.inf, -np.inf], np.nan, inplace=True)
    print(dataset['listed_count'].mean())
    print("Before fill: Does the dataset contain NaN value? {}".format(np.any(np.isnan(dataset['listed_count']))))
    dataset['listed_count'].fillna(0,inplace=True)
    print("After fill: Does the dataset contain NaN value? {}".format(np.any(np.isnan(dataset['listed_count']))))

1.9174988197099399
Before fill: Does the dataset contain NaN value? True
After fill: Does the dataset contain NaN value? False


In [397]:
df_valid.drop(df_valid[df_valid['word_count'] < 10].index, inplace=True)
df_valid.shape

(328, 34)

In [398]:
df_valid.reset_index(inplace=True, drop=True)

In [401]:
# df_valid

# Word2vec

### Load Word Vector model and Vectorize the sentence

In [402]:
word2vec_ = w2v.Word2Vec.load('w2v_model')

In [403]:
word_vectors = word2vec_.wv
vocabs = word_vectors.vocab.keys()

In [404]:
w2v_vectors = word_vectors.vectors # here you load vectors for each word in your model
w2v_indices = {word: word_vectors.vocab[word].index for word in word_vectors.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

In [405]:
# word2vec_.most_similar('liniers')

In [406]:
def vectorize(line): 
    words = []
    
    for word in line: # line - iterable, for example list of tokens 
        try:
            w2v_idx = w2v_indices[word]
        except KeyError: # if you does not have a vector for this word in your w2v model, continue 
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)

        if len(line) > len(words):
            continue
    return np.asarray(words)

## Average of Vectors & Previous features

In [407]:
import copy
df_valid['text_token_vec'] = copy.deepcopy(df_valid['text_token'])
print(df_valid.shape)

for index, sentence in enumerate(df_valid['text_token_vec']):
    df_valid['text_token_vec'][index] = vectorize(sentence).mean(axis=0)
print(df_valid.shape)


# df_valid_avg = pd.DataFrame(df_valid_avg.values.tolist()).add_prefix('token_avg') #.join(df)
# df_valid_avg = pd.DataFrame(df_valid_avg.token_avg0.tolist())

# df_valid[['text_token','text_token_vec']].head()
avg = pd.DataFrame(df_valid['text_token_vec'].values.tolist()).add_prefix('token_avg') #.join(df)

df_valid_avg = df_valid.join(avg).drop('text_token_vec',axis=1)
df_valid.drop(['text_token_vec'],axis=1,inplace=True)

(328, 35)
(328, 35)


In [408]:
print(df_valid.isna().sum())
# df_valid.dropna(inplace=True)

text                     0
hasURL                   0
urls                     0
urls_expanded            0
hasUserURL               0
user_url               119
text_token               0
isNotOnlyText            0
Noun                     0
Verb                     0
Adjective                0
Pronoun                  0
FirstPersonPronoun       0
SecondPersonPronoun      0
ThirdPersonPronoun       0
Adverb                   0
Numeral                  0
Conjunction_inj          0
Particle                 0
Determiner               0
Modal                    0
Whs                      0
char_count               0
word_count               0
has_question             0
has_exclaim              0
has_period               0
capital_ratio            0
tweet_count              0
listed_count             0
follow_ratio             0
age                      0
verified                 0
isRumor                  0
dtype: int64


# Doc2Vec

## Infer the document vectors from trained Doc2Vec Model

In [409]:
Doc2vec_model = Doc2Vec.load('./d2v_model')

### Train data

In [410]:
df_valid['text_token_doc'] = copy.deepcopy(df_valid['text_token'])

for index, sentence in enumerate(df_valid['text_token_doc']):
    df_valid['text_token_doc'][index] = Doc2vec_model.infer_vector(df_valid['text_token_doc'][index],steps=50)

# df_test_X[['text_token','text_token_doc']].head()
doc = pd.DataFrame(df_valid['text_token_doc'].values.tolist()).add_prefix('doc_vec')

df_valid_doc = df_valid.join(doc).drop('text_token_doc',axis=1)
# df_train_X_doc2.drop('text_token_doc', axis=1, inplace=True)

# Exporting data

In [411]:
# X train: df_train_X -> df_train_X_doc | df_train_X_avg -> 둘다 text/text_token을 드랍해야함
# y train: df_train_y
# X test: df_test_X -> df_test_X_doc | df_test_X_avg
# y test: df_test_y

In [412]:
df_valid_avg.to_csv('./valid_avg.csv', index = False) # '''Word2Vec Averaged Vector Feature set'''
df_valid_doc.to_csv('./valid_doc.csv', index = False) # '''Doc2vec Vector Feature set'''

In [413]:
# '''Export data without word vector'''
# data = pd.concat([df_train_X, df_test_X], ignore_index=True)
# data2 = pd.concat([df_train_y, df_test_y], ignore_index=True)

# data.to_csv('./X_basic.csv', index = False)
# data2.to_csv('./y_basic.csv', index = False)